<a href="https://colab.research.google.com/github/LongS1eeper/ML-DL_Prac/blob/%ED%98%BC%EA%B3%B5/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0_05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

## 검증 세트
* 테스트 세트로 계속 테스트 한다면 이 테스트 세트에 과대적합되는 것이 아닌가 ?
* 에 대한 문제를 해소하기 위해 80%의 훈련세트에서 20%를 검증세트로 분류

In [46]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [47]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()  # 독립변수
target = wine['class'].to_numpy()     # 종속변수

In [48]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [49]:
# 훈련세트를 다시 훈련세트 60%와 검증세트 20%로 분류
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [50]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [51]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))    # 과대적합 ?
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증
* 검증세트를 만드느라 훈련세트가 줄어든 상황
* **3-fold 교차 검증**은 훈련세트1, 훈련세트2, 검증세트2 총 3 부류의 세트로 구성
* 두세트로 훈련 후 남은 세트로 모델 평가하는 방식 3번 반복
* 이런 교차검증을 통해 줄어든 훈련세트로 인한 결과값 정확도 하락 방지

In [52]:
from sklearn.model_selection import cross_validate    # 교차검증 함수

scores = cross_validate(dt, train_input, train_target)
print(scores)   # fit_time : 모델 훈련시간, score_time: 모델 검증시간, test_score: 최종 점수

{'fit_time': array([0.01364374, 0.01509905, 0.01365519, 0.01350713, 0.01429343]), 'score_time': array([0.00275755, 0.00140262, 0.00152421, 0.00166845, 0.00150657]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [53]:
import numpy as np

print(np.mean(scores['test_score']))    # 검증 폴드의 점수 평균 구하기

0.855300214703487


* 위쪽 분석은 이전에 전체 데이터를 섞어 훈련세트를 준비한 후 진행했음
* 교차 검증을 처음 하려면 훈련세트를 섞어야하며 KFold 분할기 사용

In [54]:
from sklearn.model_selection import StratifiedKFold   # StratifiedKFold 함수 사용

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) # 5개로 분리
print(np.mean(scores['test_score']))

0.855300214703487


* 만약 훈련세트를 섞은 후 10-폴드 교차검증을 하고 싶은 경우 ?


In [55]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

* 하이퍼파라미터: 모델이 학습할 수 없어서 사용자가 지정해야만하는 파라미터
* 최적의 파라미터 값을 찾아야하는데 파라미터 개수가 늘어나면 사람이 찾기 너무 어려워짐
* 이걸 자동화 하기 위해 **그리스 서치** 이용
* GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차검증 모두 수행

In [56]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# min_impority_decrease 값을 조정해가며 뭐가 제일 좋은지 찾아보기 위해 딕셔너리 형성

In [57]:
# 객체 형성 하자마자 값 대입
# n-jobs는 병렬실행에 사용할 CPU 코어 수 지정, 기본 1, -1 은 모든 코어 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [58]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

* 그리드 서치는 훈련이 끝나면 교차검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델 훈련
* 그 모델이 best_estimator_에 저장됨

In [59]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


가장 좋은 파라미터 값도 best_params_에 저장

In [60]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


* 교차 검증의 평균 점수는 mean_test_score에 저장
* 5번 돌렸으니 5개가 출력

In [61]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [62]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])   # argmax = 가장 큰 점수의 인덱스 값 추출
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 먼저 탐색할 매개변수 지정 **(위에서 min_impurity_decrease)**
2. 훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음. <br>이 조합은 그리드 서치 객체에 저장 **(best_params_)**
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련. <br>이 모델도 그리드 서치 객체에 저장됨 **(best_estimator_)**

아래는 매개변수를 늘려서 실행

In [63]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),  # 불순도 최소 감소량
          'max_depth': range(5, 20, 1),                               # 트리 깊이 제한
          'min_samples_split': range(2, 100, 10)                      # 최소 샘플 수
          }

In [64]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [65]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [66]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치
* 위의 min_impurity_decrease 처럼 매개변수 값이 수치일 때 범위나 간격을 정하기 어려울 수 있음
* 매개변수가 많으면 그리드 수행 서치 기간이 길어질 수 있음
* 랜덤서치는 매개변수를 샘플링할 수 있는 확률분포객체를 전달

In [67]:
# randint -> 균등분포에서 정수 샘플링
# uniform -> 균등분포에서 실수 샘플링
from scipy.stats import uniform, randint

In [68]:
rgen = randint(0, 10)     # 0부터 9까지 정수를 무작위로 뽑는 "분포 객체" 생성
rgen.rvs(10)              # 정수 10개를 샘플링해서 리스트로 반환

array([3, 3, 4, 2, 6, 2, 5, 0, 6, 5])

In [69]:
np.unique(rgen.rvs(1000), return_counts=True)   # 0이 94개, 1이 105개

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 94, 105, 100,  92, 104, 100,  97, 102,  96, 110]))

In [70]:
ugen = uniform(0, 1)      # 0부터 1까지 실수를 무작위로 뽑음
ugen.rvs(10)              # 정수 10개를 샘플링해서 리스트로 반환

array([0.29035432, 0.37396682, 0.26322352, 0.33327667, 0.03141348,
       0.07408048, 0.63750143, 0.31703436, 0.01285498, 0.00793003])

* 매개변수 딕셔너리 형성


In [71]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),     # 리프노드가 되기 위한 최소 샘플 개수
          }

In [78]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),  # 모델: 결정트리
    params,                                   # 실험할 파라미터들 (dict 형태)
    n_iter=100,                               # 100번 무작위 조합을 실험
    n_jobs=-1,                                # CPU 여러 개 동시에 사용 (병렬 처리)
    random_state=42                           # 랜덤 고정 (결과 재현 가능)
)
# RandomizedSearchCV는 기본적으로 5폴드교차검정이므로 5폴드로 진행됨 (cv로 지정)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d5aa7c41650>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7d5aa913ad90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d5aa7c42f10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d5aa7c43810>},
                   random_state=42)

In [73]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [74]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [75]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [76]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
# splitter 기본이 best인데 이걸 random으로 바꿔봄
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d5aa7c41650>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7d5aa913ad90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d5aa7c42f10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d5aa7c43810>},
                   random_state=42)

In [77]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': np.float64(0.00011407982271508446), 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
